In [1]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

ratings = pd.read_csv(r"C:\Users\yashn\Desktop\Projects\ML Project Dataset\Movie Recommender\ratings.csv",sep=",")
movies = pd.read_csv(r"C:\Users\yashn\Desktop\Projects\ML Project Dataset\Movie Recommender\movies.csv",sep=",")
tmdb = pd.read_csv(r"C:\Users\yashn\Desktop\Projects\ML Project Dataset\Movie Recommender\Calculate_TMDB_Ratings.csv")

ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [3]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler

class Preprocess_TMDB():
    def __init__(self,dataset=None):
        self.dataset = dataset
    
    def null_values(self):
        self.dataset[['Average_Vote_TMDB', 'Vote_Count_TMDB','Popularity']] = self.dataset[['Average_Vote_TMDB','Vote_Count_TMDB','Popularity']].fillna(value=0)
        self.dataset[['Adult']] = self.dataset[['Adult']].fillna(value="FALSE")
        self.dataset[['Collection']] = self.dataset[['Collection']].fillna(value="None")
    
    def dropping_col(self,*args):
        k = (self.dataset).columns
        for i  in args:
            if i in k:
                self.dataset =( self.dataset).drop(i,axis = 1)
            else:
                print("No Feature Named",'"',str(i),'"')
    
    def normalize(self,*args):
        k = (self.dataset).columns
        for i  in args:
            if i in k:
                scaler = RobustScaler()
                self.dataset[i] = scaler.fit_transform(self.dataset[i].values.reshape(-1, 1))
            else:
                print("No Feature Named",'"',str(i),'"') 
    
    def dv(self,*args):
        lst = []
        k = (self.dataset).columns
        for i  in args:
            if i in k: 
                self.dataset[i] = self.dataset[i]/10
            else:
                print("No Feature Named",'"',str(i),'"')
    def minmax(self,*args):
        k = (self.dataset).columns
        for i  in args:
            if i in k:
                scaler = MinMaxScaler()
                self.dataset[i] = scaler.fit_transform(self.dataset[i].values.reshape(-1, 1))
            else:
                print("No Feature Named",'"',str(i),'"') 
    
    def fit(self,data):
        self.dataset = data
        
    def collection_encoding(self):
        lst = self.dataset['Collection'].values
        dct = {}
        sol = []
        count = 1
        for i in lst:
            if i == "None":
                sol.append(count)
                count+=1
            elif i in dct:
                sol.append(dct[i])
            else:
                dct[i] = count
                sol.append(count)
                count+=1
    
        self.dataset['Collection'] = sol      
        
    
        
    def run(self):
        col = self.dataset.columns
        req = ['Average_Vote_TMDB','Vote_Count_TMDB','Popularity','movieId','Collection','Adult']
        self.null_values()
        self.dv('Average_Vote_TMDB')
        self.minmax('Vote_Count_TMDB','Popularity')
        self.collection_encoding()
        for i in col:
            if i  not in req:
                self.dropping_col(i)
    def ret(self):
        return self.dataset
    
t = Preprocess_TMDB()
t.fit(tmdb)
t.run()
tmdb = t.ret()

    
        

In [4]:
tmdb

,movieId,Average_Vote_TMDB,Vote_Count_TMDB,Popularity,Adult,Collection
0,1,0.80,0.488280,0.247084,FALSE,1
1,2,0.72,0.284727,0.017516,FALSE,2
2,3,0.65,0.009338,0.008257,FALSE,3
3,4,0.63,0.003975,0.011291,FALSE,4
4,5,0.62,0.018204,0.016019,FALSE,5
...,...,...,...,...,...,...
9737,193581,0.76,0.002208,0.008140,FALSE,8795
9738,193583,0.79,0.008897,0.054266,FALSE,8796
9739,193585,0.68,0.000505,0.002189,FALSE,8797
9740,193587,0.81,0.002934,0.047760,FALSE,8798


In [5]:
class Preprocess_Movies():
    def __init__(self,dataset=None,other_dataset = None):
        self.dataset = dataset
        self.other_dataset = other_dataset
    
    def genre_split(self):
        self.dataset['genres'] = self.dataset['genres'].str.split('|')
        
        dfx = pd.get_dummies(pd.DataFrame(self.dataset['genres'].tolist()).stack()).sum(level=0)      
        self.dataset['year'] = self.dataset['title'].str.extract('.*\((.*)\).*',expand = False)
        self.dataset['genres_vector']= dfx.values.tolist()
        self.dataset = self.dataset.drop("genres",axis=1)
        
        
    def fit(self,dataset1,dataset2):
        self.dataset = dataset1
        self.other_dataset = dataset2
    
    def run(self):
        self.genre_split()
        self.dataset = self.dataset.merge(self.other_dataset,on='movieId', how='left')
        
    def ret(self):
        return self.dataset

t = Preprocess_Movies()
t.fit(movies,tmdb)
t.run()
movies = t.ret()       
#ratings = ratings.merge(movies,on='movieId', how='left')
#ratings  

In [6]:
movies

,movieId,title,year,genres_vector,Average_Vote_TMDB,Vote_Count_TMDB,Popularity,Adult,Collection
0,1,Toy Story (1995),1995,"[0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.80,0.488280,0.247084,FALSE,1
1,2,Jumanji (1995),1995,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.72,0.284727,0.017516,FALSE,2
2,3,Grumpier Old Men (1995),1995,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.65,0.009338,0.008257,FALSE,3
3,4,Waiting to Exhale (1995),1995,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.63,0.003975,0.011291,FALSE,4
4,5,Father of the Bride Part II (1995),1995,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.62,0.018204,0.016019,FALSE,5
...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),2017,"[0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.76,0.002208,0.008140,FALSE,8795
9738,193583,No Game No Life: Zero (2017),2017,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.79,0.008897,0.054266,FALSE,8796
9739,193585,Flint (2017),2017,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.68,0.000505,0.002189,FALSE,8797
9740,193587,Bungo Stray Dogs: Dead Apple (2018),2018,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.81,0.002934,0.047760,FALSE,8798


In [7]:
ratings.to_csv(r'C:\Users\yashn\Desktop\ratings_pred.csv')
movies.to_csv(r'C:\Users\yashn\Desktop\movies_pred.csv')

In [69]:
from scipy import spatial
from math import sqrt
class Movie_Pred():
    def __init__(self,userId=1,ratings=None, movies=None):
        self.user = userId
        self.ratings = ratings
        self.movies = movies
        self.seen = None  # lst to store all seen movies of user
        self.best = None  # lst to store best movies of user
        self.mov_score = None  # dct To store movie ratings calculated, as per movie
        self.user_score = None #dct to store movie ratings based on users
        self.fin_scores = None #dct to store final scores
        self.user_rat = None #dct to store ratings of top movies
        
    def user_best_mov(self):
        to_be_taken = 10
        dk = self.ratings.copy()
        dk = dk[dk["userId"]==self.user]
        self.seen = dk['movieId'].values
        dk = dk.sort_values(by=['rating','timestamp'],ascending=False)
        self.best = dk.head(to_be_taken)['movieId'].values
        count = 0
        dct = {}
        for index,row in dk.iterrows():
            
            if (count==to_be_taken):
                break
            dct[row['movieId']] = row['rating']
        self.user_rat = dct
        
            

    def distance(self, a, b):
        genresA = a['genres_vector']
        genresB = b['genres_vector']
        score = 1-spatial.distance.cosine(genresA, genresB)
        if a['Collection'] == b['Collection']:
            score+=1
        return score
    
    def Calculate_Movies(self):
        dct = {}
        best_data = self.movies.loc[self.movies['movieId'].isin(self.best)]
        for index, row in self.movies.iterrows():
            if row['movieId'] not in self.seen:                
                score = (row['Average_Vote_TMDB']*row['Vote_Count_TMDB']) + row['Popularity']
                for index2,row2 in best_data.iterrows():
                    score += self.distance(row,row2)*(self.user_rat[row2['movieId']]/5)
                dct[row['movieId']] = score
        self.mov_score = dct
    
    
    
    def user_recommendation(self):
        dct = {}
        dk = self.ratings[self.ratings["userId"]==self.user]
        users = self.ratings[self.ratings['movieId'].isin(dk['movieId'].tolist())]
        users = users[users['userId']!=self.user]
        subset = users.groupby(['userId'])
        subset = sorted(subset,  key=lambda x: len(x[1]), reverse=True)
        subset = subset[0:100]
        pearsonCorDict = {}
        users = users.sort_values(by='movieId')        
        for name, group in subset:
            group = group.sort_values(by='movieId')
            n = len(group) 
            temp = users[users['movieId'].isin(group['movieId'].tolist())]    
            tempRatingList = temp['rating'].tolist()
            tempGroupList = group['rating'].tolist()    
            Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
            Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
            Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)
            if Sxx != 0 and Syy != 0:
                pearsonCorDict[name] = -Sxy/sqrt(abs(Sxx*Syy))
            else:
                pearsonCorDict[name] = 0
        
        pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
        pearsonDF.columns = ['similarityIndex']
        pearsonDF['userId'] = pearsonDF.index
        topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
        user_ratings=topUsers.merge(ratings, left_on='userId', right_on='userId', how='inner')
        user_ratings['weightedRating'] = user_ratings['similarityIndex']*user_ratings['rating']
        recommend = user_ratings.groupby('movieId').sum()[['similarityIndex','weightedRating']]
        recommend.columns = ['sum_similarityIndex','sum_weightedRating']
        recommend['recommendation score'] = recommend['sum_weightedRating']/recommend['sum_similarityIndex']
        recommend = recommend.drop(['sum_similarityIndex','sum_weightedRating'],axis = 1)
        
        recommend = recommend.sort_values(by='recommendation score', ascending=False)
        for index,row in recommend.iterrows():
            dct[index] = row[0]
            
        self.user_score = dct
        
        
              
    
    def predict_movies(self):
        
        dct_fin = {}
        for i in self.mov_score:
            score = self.mov_score[i]
            if i in self.user_score:
                score += self.user_score[i]
            dct_fin[i] = score
        dct_fin = sorted(dct_fin.items(), key=lambda x:x[1],reverse = True)
        dct_fin = dict(dct_fin)
        self.final_scores = dct_fin
        return dct_fin
        
    
    
        
    
    def fit(self,userId,ratings,movies):
        self.user = userId
        self.ratings = ratings
        self.movies = movies
        self.user_best_mov()
        
        self.Calculate_Movies()
        self.user_recommendation()
        return self.predict_movies()
        
       
t = Movie_Pred()
useri = int(input("Enter UserID: "))
n = int(input("Enter Number of Movies: "))
fin = t.fit(useri,ratings,movies)

#print(fin2)
count = 0
lst = []
for i in fin:
    if (count>n):
        break
    lst.append(i)
    count+=1

best_data = movies.loc[movies['movieId'].isin(lst)]
print("Our Recommendations:")
for index,row in best_data.iterrows():
    print(row[1])
   




Enter UserID: 324
Enter Number of Movies: 10
Our Recommendations:
Smoke (1995)
Eat Drink Man Woman (Yin shi nan nu) (1994)
Pulp Fiction (1994)
Night on Earth (1991)
Butcher Boy, The (1997)
My Life in Pink (Ma vie en rose) (1997)
Son of the Bride (Hijo de la novia, El) (2001)
Miracles - Mr. Canton and Lady Rose (1989)
Strictly Sexual (2008)
Love Exposure (Ai No Mukidashi) (2008)
Rough Night (2017)


In [68]:
dg = ratings.merge(movies,on='movieId', how='left')
dg1 = dg.loc[dg['userId']==useri][['rating','title']]

print("Original Ratings given by User:")
dg1

Original Ratings given by User:


,rating,title
50179,2.0,City Slickers II: The Legend of Curly's Gold (...
50180,5.0,Fargo (1996)
50181,4.0,"Godfather, The (1972)"
50182,3.0,"Princess Bride, The (1987)"
50183,3.0,"Thomas Crown Affair, The (1968)"
50184,3.0,Time Bandits (1981)
50185,4.0,Godzilla 2000 (Gojira ni-sen mireniamu) (1999)
50186,2.0,"Crew, The (2000)"
50187,4.0,Nurse Betty (2000)
50188,4.0,Almost Famous (2000)
